**提供资料：**

数据所在路径：https://www.kaggle.com/datasets/chiragsoni/ferdata?resource=download&select=train

copy api command on kaggle：
!kaggle datasets download -d chiragsoni/ferdata


步骤：数据处理-定义模型-设置损失函数和优化器-模型训练-模型评估

# 1.数据处理

In [1]:
!kaggle datasets download -d chiragsoni/ferdata

Dataset URL: https://www.kaggle.com/datasets/chiragsoni/ferdata
License(s): CC0-1.0
 93% 56.0M/60.3M [00:00<00:00, 61.9MB/s]
100% 60.3M/60.3M [00:00<00:00, 67.9MB/s]


In [ ]:
!unzip ferdata.zip -d /content/ferdata

In [3]:
train_dir='/content/ferdata/train'
test_dir='/content/ferdata/test'

In [4]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [5]:
train_transform=transforms.Compose([
    transforms.Resize([48,48]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )

])
test_transform=transforms.Compose([
    transforms.Resize([48,48]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )

])

In [6]:
train_data=ImageFolder(root=train_dir,transform=train_transform)
test_data=ImageFolder(root=train_dir,transform=test_transform)

In [7]:
train_loader=DataLoader(train_data,batch_size=16,shuffle=True,num_workers=1)
test_loader=DataLoader(test_data,batch_size=16,shuffle=True,num_workers=1)

# 2.定义模型

In [8]:
device="cuda" if torch.cuda.is_available() else "cpu"
print("using {} device ".format(device))

using cpu device 


In [9]:
from os import pread
from torchvision import models
model=models.vgg16(pretrained=True)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 106MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 3.定义损失函数和优化器

In [10]:
from torch import nn

In [11]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001,weight_decay=0.0001)

# 4.定义训练函数

In [12]:
def train(dataloader,model,loss_fn,optimizer):
  model=model.to(device)
  model.train()
  for i,(X,y) in enumerate(dataloader):
    X,y=X.to(device),y.to(device)
    pred=model(X)
    loss=loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i%1000==0):
      print("[%5d] loss:%.3f" %(i,loss))

def test(dataloader,model,loss_fn):
  size=len(dataloader.dataset)
  num_batches=len(dataloader)
  model.eval()

  test_loss,correct=0,0
  with torch.no_grad():
    for (X,y) in enumerate(dataloader):
      X,y=X.to(device),y.to(device)
      pred=model(X)
      test_loss+=loss_fn(pred,y).item()
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss/=num_batches
  correct/=size
  print("Acc:%f,loss:%f"% (correct,test_loss))

# 5.训练

In [ ]:
epoches=3
for epoch in range(epoches):
  print("---------------epoch:%d-----------------------" % epoch)
  train(train_loader,model,loss_fn,optimizer)
  test(test_loader,model,loss_fn)

---------------epoch:0-----------------------
[    0] loss:35.678
